# Notacao IOB para NER (preprocess 2/2)

Notebook para incluir labels de entidades no esquema IOB para cada trecho de texto extraido em extract_text.ipynb

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
import math

## Parte 1 - Carregamento dos dados

In [2]:
# data = pd.read_csv("verif_data.csv")
data = pd.read_csv("verif_dataV1.csv")
data['labels'] = ""

In [3]:
data.columns

Index(['REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF', 'TIPO_DODF',
       'ATO', 'COD_EMPRESA', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'COD_MATRICULA_SIGRH', 'CPF', 'NOME_ATO', 'NOME_SIGRH', 'CARGO',
       'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL', 'text',
       'labels'],
      dtype='object')

Colunas a serem utilizadas: 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO', 'CPF', 'NOME_ATO', 'CARGO', 'CLASSE',                                   'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL'

Colunas descartadas: 'Unnamed: 0', 'REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF',
       'TIPO_DODF', 'COD_EMPRESA', 'COD_MATRICULA_SIGRH', 'NOME_SIGRH'

In [4]:
data = data.drop(columns=[ 
    'REF_ANOMES', 
#     'DATA_DODF', 
    'NUM_DODF', 
    'PAGINA_DODF', 
    'TIPO_DODF', 
    'COD_EMPRESA', 
    'COD_MATRICULA_SIGRH', 
    'NOME_SIGRH', 
    'CPF'
])

In [5]:
data.columns

Index(['DATA_DODF', 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO', 'NOME_ATO',
       'CARGO', 'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL', 'text',
       'labels'],
      dtype='object')

In [6]:
print(data['FUND_LEGAL'][0], '\n\n')

# for row in range(len(data)):
#     for col in data.drop(columns=['text', 'labels']).columns:
#         if pd.notna(data[col][row]):
#             data.loc[row, col] = str(data.loc[row, col]).replace('º', 'o')

for row in range(len(data)):
    for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns:
        if pd.notna(data.loc[row, col]):
            data.loc[row, col] = unicodedata.normalize('NFKD', str(data.loc[row, col])).encode('ascii', 'ignore').decode('utf8')

print(data['FUND_LEGAL'][0])

NOS TERMOS DO ARTIGO 3º, INCISOS I, II E III, E PARAGRAFO UNICO DA EMENDA CONSTITUCIONAL N.º 47 DE 05/07/2005, COMBINADO COM O ARTIGO 44 DA LEI COMPLEMENTAR N.º 769, DE 30/06/2008, COM A VANTAGEM PESSOAL PREVISTA NO ARTIGO 5º DA LEI Nº 4.584, DE 08/07/2011 


NOS TERMOS DO ARTIGO 3o, INCISOS I, II E III, E PARAGRAFO UNICO DA EMENDA CONSTITUCIONAL N.o 47 DE 05/07/2005, COMBINADO COM O ARTIGO 44 DA LEI COMPLEMENTAR N.o 769, DE 30/06/2008, COM A VANTAGEM PESSOAL PREVISTA NO ARTIGO 5o DA LEI No 4.584, DE 08/07/2011


## Parte 2 - Funcoes para identificar entidades e anotar no padrao IOB

In [7]:
def get_regex(data, col, idx):
    entity = str(data.loc[idx, col])
    re_entity = ""
    if col == "COD_MATRICULA_ATO":
        while entity[0] == '0':
            entity = entity[1:]
        re_entity += '[0oO]*?'
    for i in entity:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_entity += f"[{i.lower()}{i.upper()}]" + "[-,.\s\"]*?"
        elif i == '(' or i == ')':
            re_entity += '\\' + i + "[-,.\s\"]*?"
        else:
            re_entity += i + "[-,.\s\"]*?"
    return re_entity

In [8]:
def IOBify_text(text, entities):
    labels = ["O" for _ in range(len(text.split()))]
    word_start_position = []
    number_of_word = []
    for i in range(len(text)):
        if (i == 0):
            if (text[i] == ' '):
                continue
            else:
                number_of_word.append(len(word_start_position))
                word_start_position.append(i)
        elif text[i] != ' ' and text[i-1] == ' ':
            number_of_word.append(len(word_start_position))
            word_start_position.append(i)
    i = 0
    for entity in entities:
        entity_begin = entity_end = -1
        # Find initial position of entity
        if entity[0] in word_start_position:
            entity_begin = word_start_position.index(entity[0])
        # Find final position of entity
        for pos, idx in zip(word_start_position, number_of_word):
            if pos > entity[1]:
                entity_end = idx-1
                break
        if entity_end == -1:
            entity_end = number_of_word[-1]
            
        if entity_begin != -1:
            for i in range(entity_begin, entity_end+1):
                if i==entity_begin:
                    labels[i] = "B-"+entity[2]
                else:
                    labels[i] = "I-"+entity[2]
    return labels
                
def find_entities(data, idx):
    list_entities = []
    for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns:
        if pd.notna(data[col][idx]):
            re_entity = get_regex(data, col, idx)
            aux = re.search(re_entity, data['text'][idx])
            if aux:
                found_entities[col]  += 1
                list_entities.append([aux.span()[0], aux.span()[1], col])
            else:
                missed_entities[col][0] += 1
                missed_entities[col][1].append(idx)
    return list_entities


## Parte 3 - Identificando entidades e anotando-as

In [9]:
found_entities  = {col: 0 for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns}
missed_entities = {col: [0, []] for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns}

for row in range(len(data)):
    if pd.notna(data.loc[row, 'text']):
        entities = find_entities(data, row)
        entities.sort()
        labels = IOBify_text(data.loc[row, 'text'], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data.loc[row, 'labels'] = s

for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

Found ATO: 5167 		 Missed ATO: 2
Found EMPRESA_ATO: 2206 		 Missed EMPRESA_ATO: 2963
Found COD_MATRICULA_ATO: 5157 		 Missed COD_MATRICULA_ATO: 12
Found NOME_ATO: 5165 		 Missed NOME_ATO: 4
Found CARGO: 5162 		 Missed CARGO: 6
Found CLASSE: 2383 		 Missed CLASSE: 39
Found PADRAO: 4873 		 Missed PADRAO: 6
Found QUADRO: 4908 		 Missed QUADRO: 44
Found PROCESSO: 5169 		 Missed PROCESSO: 0
Found FUND_LEGAL: 4568 		 Missed FUND_LEGAL: 600


## Parte 4 - Corrigindo entidades nao identificadas

In [10]:
# Carregamento dos dados originais anotados
data_og = pd.read_csv('TCDF_data/Atos_Aposentadoria_validados.csv')

In [11]:
print("Entidades nao identificadas e as linhas do dataframe em que elas aparecem:\n")
for col in data.drop(columns=['text', 'labels']).columns:
#     print(f"Col:{col} -> {missed_entities[col][1]}")
#     if len(missed_entities[col][1]) <= 100:
    if col == 'QUADRO':
        print(f"Col:{col} -> {missed_entities[col][1]}") 

Entidades nao identificadas e as linhas do dataframe em que elas aparecem:

Col:QUADRO -> [104, 138, 269, 270, 271, 272, 273, 280, 282, 526, 528, 529, 530, 531, 532, 571, 975, 976, 977, 978, 980, 981, 983, 984, 985, 987, 988, 3855, 3927, 4474, 4496, 4505, 4509, 4527, 4529, 4531, 4532, 4533, 4545, 4550, 4571, 5476, 5477, 5512]


### Correcao manual de trechos de texto identificados erroneamente

Trechos de texto que nao condizem com as entidades anotadas sao corrigidos aqui

In [12]:
# Matricula(DONE) | NOME(DONE) | CARGO(DONE) | CLASSE(DONE) | PADRAO(DONE) | QUADRO() | 

# Fix row 138
data.loc[138, 'text'] = "CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: MARIA AUXILIADORA PE- REIRA, matricula 101.008-5, processo SEI n 00070- 00011305/2018-23, no cargo de Au- xiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 6, incisos I, II, III e IV, da EC n 41/ 2003, combinado com o artigo 2 da EC n 47/2005, c/c a Lei Complementar no 769/08."

# Fix row 481
data.loc[481, 'text'] = "CONCEDER APOSENTADORIA a IZAAC NEWTON DA SILVA, matricula 38.950-1, no Cargo de Professor de Educacao Basica, Padrao 22, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00033434/2017-54."

# Fix row 506
data.loc[506, 'text'] = "CONCEDER aposentadoria voluntaria com proventos integrais ao servidor a seguir nominado: AMISAEL GONCALVES BI- NACETT, matricula 100.453-0, processo SEI n 00070-00013097/2018-05, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008."
data.loc[506, 'PROCESSO'] = "00070-00013097/2018-05"
data.loc[506, 'CARGO'] = "Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria"

# Fix row 537
data.loc[537, 'text'] = "CONCEDER APOSENTADORIA a VERA LUCIA FERREIRA DE SOUSA, matricula 40.787-9, no Cargo de Agente de Gestao Educacional/Servicos Gerais, Nivel 10, Padrao 02, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o da Emenda Constitucional no 47, de 06 de julho de 2005 e o Paragrafo unico do mesmo artigo. Processo 0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11 ."
data.loc[537, 'PROCESSO'] = "0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11"

#Fix row 571
data.loc[571, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."

# Fix row 2763
data.loc[2763, 'text'] = "APOSENTAR MARIA ESTER BATISTA DE MEDEIROS, matricula 204.859-0, no Cargo de Professor de Educacao Basica, Padrao 19, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00040-00015628/2019-89."

# Fix row 5512
data.loc[5512, 'text'] = "APOSENTAR, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."


### Correcao manual das entidades de MATRICULA

utilizando as matriculas SIGRH quando matriculas ATO nao sao encontradas

In [13]:
print(data.loc[138, 'COD_MATRICULA_ATO'], data_og.loc[138, 'COD_MATRICULA_SIGRH'])

#Fix row 138
data.loc[138, 'COD_MATRICULA_ATO'] = data_og.loc[138, 'COD_MATRICULA_SIGRH']

#Fix row 422
data.loc[422, 'COD_MATRICULA_ATO'] = data_og.loc[422, 'COD_MATRICULA_SIGRH']

#Fix row 506
data.loc[506, 'COD_MATRICULA_ATO'] = data_og.loc[506, 'COD_MATRICULA_SIGRH']

#Fix row 537
data.loc[537, 'COD_MATRICULA_ATO'] = data_og.loc[537, 'COD_MATRICULA_SIGRH']

#Fix row 719
data.loc[719, 'COD_MATRICULA_ATO'] = data_og.loc[719, 'COD_MATRICULA_SIGRH']

# FIx row 761
data.loc[761, 'COD_MATRICULA_ATO'] = data_og.loc[761, 'COD_MATRICULA_SIGRH']

# Fix row 3199
data.loc[3199, 'COD_MATRICULA_ATO'] = "209.913-6"

# Fix row 3610
data.loc[3610, 'COD_MATRICULA_ATO'] = data_og.loc[3610, 'COD_MATRICULA_SIGRH']

# Fix row 4449
data.loc[4449, 'COD_MATRICULA_ATO'] = "42.299-1"

print(data.loc[138, 'COD_MATRICULA_ATO'], data_og.loc[138, 'COD_MATRICULA_SIGRH'])

01793403 01010085
01010085 01010085


### Correcao manual das entidades de NOME

Diferenca no processamento de acentos e apostrofos entre o DODFminer e as anotacoes

In [14]:
# No fixes needed here

# # Fix row 201
# data.loc[201, 'NOME_ATO'] = "ROSELI DE MOURA GONZALES D ALMEIDA"

# # Fix row 604
# data.loc[604, 'NOME_ATO'] = "MARIA FLORA CALVINO MARQUES"

# # Fix row 5199
# data.loc[5199, 'NOME_ATO'] = "LORIVANDA D ABADIA DOS SANTOS"

# # Fix row 5229
# data.loc[5229, 'NOME_ATO'] = "ELZA LUCIA MENDES MUNIZ"

# # Fix row 5310
# data.loc[5310, 'NOME_ATO'] = "MARIA D ARC PEREIRA"

# # Fix row 5427
# data.loc[5427, 'NOME_ATO'] = "MARCIA CRISTINA TOMAZ MULLER"

### Correcao manual das entidades de CARGO

Algumas anotacoes erroneas (apresentadas abaixo)

In [15]:
# Fix row 719
data.loc[719, 'CARGO'] = "agente de transito"

# Fix row 4141
data.loc[4141, 'CARGO'] = "Gestor em Politicas Publicas e Gestao Governamental"

### Correcao manual das entidades de CLASSE

Em grande parte diferenca entre numeros ordinais e suas escritas por extenso (1a classe == primeira classe)

In [16]:
for i in missed_entities['CLASSE'][1]:
    a = data.iloc[i]['CLASSE']
    a = a.split()
    s = ""
    if a[0].lower() == 'primeira':
        s = "1a Classe"
    elif a[0].lower() == 'segunda':
        s = "2a Classe"
    elif a[0].lower() == 'terceira':
        s = "3a Classe"
    elif a[0].lower() == 'quarta':
        s = "4a Classe"
    if s:
        data.loc[i, 'CLASSE'] = s
        
# Fix row 506
data.loc[506, 'CLASSE'] = "Classe Unica"

# Fix row 571
data.loc[571, 'CLASSE'] = "Primeira Classe"

# Fix row 947
data.loc[947, 'CLASSE'] = "Classe Unica"

# Fix row 5512
data.loc[5512, 'CLASSE'] = "Primeira Classe"


### Correcao manual das entidades de PADRAO

Entidades da classe PADRAO foram anotadas erroneamente (casos abaixo)

In [17]:
# Fix row 3199
data.loc[3199, 'PADRAO'] = "Padrao 15"

# Fix row 4449
data.loc[4449, 'PADRAO'] = "Padrao 25"

### Correcao manual das entidades de QUADRO

Ainda nao sei qual o erro

In [18]:
print("Entidades nao identificadas e as linhas do dataframe em que elas aparecem:\n")
for col in data.drop(columns=['text', 'labels']).columns:
#     print(f"Col:{col} -> {missed_entities[col][1]}")
#     if len(missed_entities[col][1]) <= 100:
    if col == 'QUADRO':
        print(f"Col:{col} -> {missed_entities[col][1]}") 

Entidades nao identificadas e as linhas do dataframe em que elas aparecem:

Col:QUADRO -> [104, 138, 269, 270, 271, 272, 273, 280, 282, 526, 528, 529, 530, 531, 532, 571, 975, 976, 977, 978, 980, 981, 983, 984, 985, 987, 988, 3855, 3927, 4474, 4496, 4505, 4509, 4527, 4529, 4531, 4532, 4533, 4545, 4550, 4571, 5476, 5477, 5512]


In [19]:
# Celula utilizada para ver quais as entidades e o trecho de texto identificado para uma linha do dataframe (row)
# [506, 537]
row = 104
print(data_og.loc[row, 'DATA_DODF'], data_og.loc[row, 'NUM_DODF'])
print(data.loc[row, 'NOME_ATO'], "|||", data['QUADRO'][row], "\n|||", data['PROCESSO'][row], "|||", data_og['NUM_DODF'][row], "|||", data_og['DATA_DODF'][row])
print(data['CLASSE'][row], "|||", data['PADRAO'][row], "|||", data['QUADRO'][row])
print()
print(data['text'][row])

10/04/2018 68
DINA MARIA PIRES DE MIRANDA ||| QUADRO DE PESSOAL DO DISTRITO FEDERAL 
||| 00361.00004000/201835 ||| 68 ||| 10/04/2018
CLASSE ESPECIAL ||| PADRAO V ||| QUADRO DE PESSOAL DO DISTRITO FEDERAL

CONCEDER Aposentadoria a DINA MARIA PIRES DE MIRANDA, matricula 25.293-X, no cargo de Auditor Fiscal de Atividades Urbanas, Classe Especial, Padrao V, do Quadro de Diario Oficial do Distrito Federal No 68, terca-feira, 10 de abril de 2018 PAGINA 20 Este documento pode ser verificado no endereco eletronico http://www.in.gov.br/autenticidade.html , pelo codigo 50012018041000020 Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a Infraestrutura de Chaves Publicas Brasileira - ICP-Brasil. Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47, de 06 de julho de 2005, combinado com o artigo 44, da Lei Complementar no 769, de 30 de junho de 2008. Processo SEI n o 00361.00004000/2018- 35


In [20]:
for i in missed_entities['QUADRO'][1]:
    if data.loc[i, 'QUADRO'] == 'QUADRO DE PESSOAL DO SERVICO DE LIMPEZA URBANA':
        data.loc[i, 'QUADRO'] = "QP/SLU"
        
# Fix row 3927 (DODFminer pulou parte do texto do DODF)
data.loc[3927, 'QUADRO'] = "Quadro de Pessoal do Governo do Distrito"

# Fix row 104 (DODFminer pulou parte do texto do DODF)
data.loc[104, 'QUADRO'] = "Quadro de"

### Correcao manual das entidades de EMPRESA_ATO

Algumas anotacoes nao aparecem no DODF

In [21]:
# rows em que entidades EMPRESA_ATO nao aparecem no DODF
# no_entity = [70, 71, ]

# for i in no_entity:
#     data.loc[i, 'EMPRESA_ATO'] = ""



### Correcao manual das entidades de FUND_LEGAL

Ainda nao sei qual o erro

In [22]:
print("Entidades nao identificadas e as linhas do dataframe em que elas aparecem:\n")
print("Numero de entidades FUND_LEGAL nao encontradas:", len(missed_entities['FUND_LEGAL'][1]))
# for col in data.drop(columns=['text', 'labels']).columns:
# #     print(f"Col:{col} -> {missed_entities[col][1]}")
# #     if len(missed_entities[col][1]) <= 100:
#     if col == 'FUND_LEGAL':
#         print(f"Col:{col} -> {missed_entities[col][1]}") 

Entidades nao identificadas e as linhas do dataframe em que elas aparecem:

Numero de entidades FUND_LEGAL nao encontradas: 600


In [23]:
# Utilizando as tres primeiras e tres ultimas palavras da entidade anotada (ao inves de a entidade completa) com regex

print(data.loc[290, 'FUND_LEGAL'])
for sample in missed_entities['FUND_LEGAL'][1]:
    s = data.loc[sample, 'FUND_LEGAL'].split()
    beg = ' '.join(s[:4])
    end = ' '.join(s[-3:len(s)])
        
    # Criando regex para a entidade FUND_LEGAL (utilizando as primeiras 3 e as ultimas 3 palavras de data.loc[sample, 'FUND_LEGAL'])
    re_FUND_LEGAL = ""
    for i in beg:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_FUND_LEGAL += f"[{i.lower()}{i.upper()}]" + "[-,.\s]*?"
        else:
            re_FUND_LEGAL += i + "[-,.\s]*?"
    re_FUND_LEGAL += '.*?'
    for i in end:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_FUND_LEGAL += f"[{i.lower()}{i.upper()}]" + "[-,.\s]*?"
        else:
            re_FUND_LEGAL += i + "[-,.\s]*?"
            
    # Busca do regex no trecho de texto que deve conter esta entidade
    result = re.search(re_FUND_LEGAL, data.loc[sample, 'text'])
    if result:
        if abs(len(data.loc[sample, 'text'][result.span()[0]:result.span()[1]].split()) - len(data.loc[sample, 'FUND_LEGAL'].split())) < 3:
            data.loc[sample, 'FUND_LEGAL'] = result[0]
            
print(data.loc[290, 'FUND_LEGAL'])


NOS TERMOS DO ARTIGO 40,  1o, INCISO III, ALINEA AE  3o, 5o,8o E 17, DA CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, NA REDACAO DADA PELA EMENDA CONSTITUCIONAL No 41, DE 31 DE DEZEMBRO DE 2003, E ARTIGOS 46 E 51 DA LEI COMPLEMENTAR No 769, DE 01 DE JULHO DE 2008
nos termos do artigo 40,  1o, inciso III, alinea "a"e 3o, 5o,8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao dada pelaEmenda Constitucional no 41, de 31 de dezembro de 2003, e artigos 46 e 51 da LeiComplementar no 769, de 01 de julho de 2008


## Parte 5 - Reanotando entidades corrigidas

Similar a Parte 3, mas agora utilizando os dados corrigidos na Parte 4

In [24]:
found_entities  = {col: 0 for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns}
missed_entities = {col: [0, []] for col in data.drop(columns=['text', 'labels', 'DATA_DODF']).columns}
data['labels'] = ""
for row in range(len(data)):
    if pd.notna(data['text'][row]):
        entities = find_entities(data, row)
        entities.sort()
        labels = IOBify_text(data['text'][row], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data['labels'][row] = s
    
for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

Found ATO: 5167 		 Missed ATO: 2
Found EMPRESA_ATO: 2208 		 Missed EMPRESA_ATO: 2961
Found COD_MATRICULA_ATO: 5169 		 Missed COD_MATRICULA_ATO: 0
Found NOME_ATO: 5169 		 Missed NOME_ATO: 0
Found CARGO: 5168 		 Missed CARGO: 0
Found CLASSE: 2422 		 Missed CLASSE: 0
Found PADRAO: 4879 		 Missed PADRAO: 0
Found QUADRO: 4952 		 Missed QUADRO: 0
Found PROCESSO: 5169 		 Missed PROCESSO: 0
Found FUND_LEGAL: 5069 		 Missed FUND_LEGAL: 99


## Final - salvando os dados preprocessados

### Salvando no formato CoNLL03 

Salva os trechos de texto e labels, separando em conjuntos "nao rotulado" e "rotulado" para teste com active learning

In [25]:
x = []
y = []
t = []

# Separa conjuntos de dados de forma cronológica
def func(a):
    a = a.split('/')
    return a[2], a[1], a[0]
for row in data.index:
    if pd.notna(data['text'][row]) and pd.notna(data['labels'][row]):
        x.append(data.loc[row, 'text'])
        y.append(data.loc[row, 'labels'])
        t.append(' '.join((lambda x: x[::-1])(data.loc[row, 'DATA_DODF'].split('/'))))
idx = np.argsort(t)
test_split_idx = math.floor(len(idx)*0.7)
t1 = [-1 for _ in range(len(t))]
for i, j in enumerate(idx):
    t1[i] = t[j]
x_train = [x[i] for i in idx[:test_split_idx]]
y_train = [y[i] for i in idx[:test_split_idx]]
x_test = [x[i] for i in idx[test_split_idx:]]
y_test = [y[i] for i in idx[test_split_idx:]]

initial_labeled_set_idx = math.floor(len(x_train)*0.99)
labeled_x = x_train[initial_labeled_set_idx:]
labeled_y = y_train[initial_labeled_set_idx:]
unlabeled_x = x_train[:initial_labeled_set_idx]
unlabeled_y = y_train[:initial_labeled_set_idx]
print("Tamanho do conjunto de treinamento")
print("unlabeled set size:", len(unlabeled_x), "labeled set size:",  len(labeled_x))
print("\nTamanho do conjunto de teste")
print("test set size:", len(x_test))

Tamanho do conjunto de treinamento
unlabeled set size: 3581 labeled set size: 37

Tamanho do conjunto de teste
test set size: 1551


In [26]:
unlabeled_x[0]

'CONCEDER aposentadoria voluntaria ao servidor MANOEL FRANCISCO DOS SANTOS, matricula no 101.644-X, Tecnico Socioeducativo - Classe Especial, Padrao V, fundamentada nos termos do artigo 3o, incisos I, II, III e paragrafo unico, da Emenda Constitucional no 47/2005, combinado com o artigo 44 da Lei Complementar no 769/2008, com as vantagens do artigo 5o da Lei no 4.584/11. Processo SEI no 00417-00013797/2018-32'

In [27]:
def save_conll(x, y, nome_arquivo):
    ptr = open(nome_arquivo, 'w')
    for i in range(len(x)):
        text = x[i]
        label = y[i]
        assert len(text.split()) == len(label.split()), f"Erro em {text}"
        for word, label in zip(text.split(), label.split()):
            ptr.write(word + ' ' + label + '\n')
        ptr.write('\n')
save_conll(labeled_x,   labeled_y,   "labeled_set.txt"  )
save_conll(unlabeled_x, unlabeled_y, "unlabeled_set.txt")
save_conll(x_test,      y_test,      "test_set.txt"     )
save_conll(x_train,     y_train,     "train_set.txt")

### Salvando o dataframe completo 

Salva dataframe contendo anotacoes de entidades, trechos de texto contendo as anotacoes e as labels (IOB) para cada palavra no texto.

In [28]:
data.to_csv('labeled.csv', index=False)